## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.common.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 4.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1/160, -1/160, 1/80, -1/80, 1/40, -1/40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2])


converged SCF energy = -14.9345493818237
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXXIIIIIIYX, +IIXXIIIIYXII, +IIXXIIYXIIII, +IIXXYXIIIIII,

## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-15.33649467289809


In [4]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -14.934549381823707


## Setup for GPT

In [5]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.n_gates = 30  # The number of gates for each circuit
model_config.block_size = model_config.n_gates
model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0.1
model_config.resid_pdrop = 0.1
model_config.attn_pdrop = 0.1
model_config.std = 0.02
model_config.energy_offset = 14
model = GPT(model_config, cost)

number of parameters: 85.79M


In [6]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 1000
train_config.num_workers = 10
train_config.n_samples = 50
trainer = Trainer(train_config, model)

running on device mps


In [7]:
file_monitor = FileMonitor()
callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor], del_temperature=0.01)
trainer.set_callback('on_batch_end', callback_generator.generate())
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

iter_dt 0.00s; iter 0: train loss 0.14985 temperature: 5
mean_logits tensor([-14.9388, -14.9528, -15.0071, -15.0647, -14.8301, -14.8985, -15.0804,
        -14.8169, -15.0770, -15.0296, -14.9578, -14.8626, -14.8198, -14.8334,
        -14.7787, -14.9138, -14.9562, -15.1092, -15.0414, -14.9065, -15.0057,
        -15.0122, -14.7064, -15.0227, -14.7585, -14.9664, -14.8754, -14.8604,
        -15.0395, -15.0138, -14.9117, -14.8076, -15.0506, -15.0247, -15.2137,
        -14.9663, -14.9443, -14.9499, -15.0470, -14.9670, -15.0601, -14.9286,
        -15.1159, -14.9852, -14.8846, -15.0560, -14.9432, -15.0407, -15.2327,
        -14.7911], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.8369, -14.8863, -14.8611, -14.9317, -14.8004, -14.7980, -15.1061,
        -14.6344, -14.9618, -14.8628, -14.8188, -14.8261, -15.0032, -14.9624,
        -14.9481, -15.0096, -14.6682, -15.0220, -14.9498, -15.0486, -14.9168,
        -14.9275, -15.0003, -14.8577, -15.0809, -14.9733, -14.8389, -14.9229,
    


KeyboardInterrupt



In [ ]:
print(file_monitor.min_energy)


In [ ]:
cost.sequence.tool = "qiskit"
print(file_monitor.min_indices)
cost.sequence._get_circuit(file_monitor.min_indices).qc.draw(output="mpl", plot_barriers=True)